Anurag Sahu


anuragsahu.2904@gmail.com

In [2]:
 !pip install streamlit
 import os
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up"],
        "responses": ["Hi there", "Hello", "Hey", "I'm fine, thank you", "Nothing much"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye", "See you later", "Take care"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
        "tag": "store_hours",
        "patterns": ["What are your store hours?", "When do you open?", "When do you close?", "Are you open on weekends?"],
        "responses": ["Our store hours are Monday to Friday from 9 AM to 9 PM and Saturday to Sunday from 10 AM to 6 PM.", "We open at 9 AM on weekdays and at 10 AM on weekends.", "We close at 9 PM on weekdays and at 6 PM on weekends."]
    },
    {
        "tag": "return_policy",
        "patterns": ["What is your return policy?", "How do I return an item?", "Can I return a purchase?"],
        "responses": ["Our return policy allows you to return items within 30 days of purchase with a receipt.", "You can return an item by bringing it to the store with your receipt.", "Yes, you can return a purchase within 30 days with a receipt."]
    },
    {
        "tag": "shipping_info",
        "patterns": ["What are your shipping options?", "How much does shipping cost?", "Do you offer free shipping?"],
        "responses": ["We offer standard, expedited, and next-day shipping options.", "Shipping costs vary depending on the option you choose and the destination.", "We offer free standard shipping on orders over $50."]
    },
    {
        "tag": "product_availability",
        "patterns": ["Is this product available?", "Do you have this in stock?", "Can I find this item in-store?"],
        "responses": ["Let me check the availability for you.", "Yes, this product is available in our store.", "This item is currently in stock and can be found in-store."]
    },
    {
        "tag": "payment_methods",
        "patterns": ["What payment methods do you accept?", "Can I pay with a credit card?", "Do you accept PayPal?"],
        "responses": ["We accept credit cards, debit cards, and PayPal.", "Yes, you can pay with a credit card.", "Yes, we accept PayPal."]
    },
    {
        "tag": "order_status",
        "patterns": ["Where is my order?", "How can I track my order?", "What is the status of my order?"],
        "responses": ["You can track your order using the tracking number provided in your confirmation email.", "To check the status of your order, please enter your order number on our order tracking page.", "Your order is currently being processed and will be shipped soon."]
    },
    {
        "tag": "discounts",
        "patterns": ["Do you have any discounts?", "Are there any sales?", "Can I get a coupon?"],
        "responses": ["We are currently offering a 20% discount on all items.", "Yes, we have a sale going on. Check our website for details.", "You can sign up for our newsletter to receive exclusive coupons."]
    },
    {
        "tag": "loyalty_program",
        "patterns": ["Do you have a loyalty program?", "How can I join your loyalty program?", "What are the benefits of your loyalty program?"],
        "responses": ["Yes, we have a loyalty program. You can join by signing up on our website.", "To join our loyalty program, please fill out the registration form on our website.", "Our loyalty program offers exclusive discounts, early access to sales, and reward points for every purchase."]
    },
    {
        "tag": "product_support",
        "patterns": ["I need help with a product.", "How do I use this product?", "Can you provide instructions for this product?"],
        "responses": ["Sure, what product do you need help with?", "You can find instructions for this product in the manual or on our website.", "I can help with that. What specific issue are you facing?"]
    }
]



In [4]:
intents.extend([
    {
        "tag": "product_recommendation",
        "patterns": ["Can you recommend a product?", "What should I buy?", "I'm looking for a gift, any suggestions?"],
        "responses": ["Sure, what type of product are you looking for?", "I can help with that. Who is the gift for and what do they like?", "Let me suggest some popular products for you."]
    },
    {
        "tag": "size_guide",
        "patterns": ["What size should I buy?", "Do you have a size guide?", "How do I know my size?"],
        "responses": ["You can refer to our size guide on the product page.", "Let me help you with the sizing. What are your measurements?", "Our size guide can help you find the right fit."]
    },
    {
        "tag": "order_cancellation",
        "patterns": ["Can I cancel my order?", "How do I cancel an order?", "I need to cancel my purchase."],
        "responses": ["You can cancel your order within 24 hours of placing it.", "To cancel your order, please visit the order status page and select 'Cancel Order'.", "I'm here to help. What's your order number?"]
    },
    {
        "tag": "store_locator",
        "patterns": ["Where is your store?", "Do you have a store near me?", "What is your store address?"],
        "responses": ["You can find our store locations on the 'Store Locator' page of our website.", "Please enter your zip code on our website to find the nearest store.", "Our store is located at [Store Address]."]
    },
    {
        "tag": "gift_cards",
        "patterns": ["Do you sell gift cards?", "Can I buy a gift card?", "How do I use a gift card?"],
        "responses": ["Yes, we offer gift cards. You can purchase them on our website.", "Gift cards are available in various denominations. Check our website for more details.", "You can use a gift card by entering the card number during checkout."]
    },
    {
        "tag": "warranty",
        "patterns": ["What is the warranty on this product?", "Do you offer warranties?", "How do I claim a warranty?"],
        "responses": ["We offer a one-year warranty on all our products.", "Yes, we provide warranties. Please check the product details for warranty information.", "To claim a warranty, please contact our customer service with your purchase details."]
    },
    {
        "tag": "product_comparison",
        "patterns": ["Can you compare these products?", "Which product is better?", "I'm deciding between two products."],
        "responses": ["Sure, I can help compare the products for you.", "Let's look at the features of both products to find the best option.", "What are the main features you're looking for?"]
    },
    {
        "tag": "stock_alerts",
        "patterns": ["Can you notify me when this product is back in stock?", "How do I get stock alerts?", "Is there a waitlist for out-of-stock items?"],
        "responses": ["You can sign up for stock alerts on the product page.", "We will notify you via email when the product is back in stock.", "Please join the waitlist on our website to receive stock updates."]
    },
    {
        "tag": "delivery_time",
        "patterns": ["How long does delivery take?", "When will my order arrive?", "What are your delivery times?"],
        "responses": ["Delivery times vary depending on your location. Generally, it takes 3-5 business days.", "You can check the estimated delivery time on your order confirmation page.", "Our standard delivery time is 3-5 business days."]
    },
    {
        "tag": "special_offers",
        "patterns": ["Do you have any special offers?", "Are there any promotions?", "What are your current deals?"],
        "responses": ["We have ongoing promotions. Please check our 'Special Offers' page.", "Yes, we have several deals running currently. Visit our website for more details.", "You can find our latest promotions on the homepage."]
    },
    {
        "tag": "customer_reviews",
        "patterns": ["Can I see customer reviews?", "What do other customers say about this product?", "Are there any reviews for this item?"],
        "responses": ["Customer reviews are available on the product page.", "Here are some reviews from other customers.", "You can read customer reviews under the 'Reviews' section on the product page."]
    },
    {
        "tag": "price_match",
        "patterns": ["Do you offer price matching?", "Can you match this price?", "How does your price match policy work?"],
        "responses": ["Yes, we offer price matching. Please provide the details of the lower price.", "We can match the price if it's from a verified competitor.", "Our price match policy requires proof of the lower price. Contact customer service for more information."]
    }
])


In [5]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

# Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

# training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [6]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [7]:
counter = 0

def main():
    global counter
    st.title("Chatbot")
    st.write("Welcome to the chatbot. Please type a message and press Enter to start the conversation.")

    counter += 1
    user_input = st.text_input("You:", key=f"user_input_{counter}")

    if user_input:
        response = chatbot(user_input)
        st.text_area("Chatbot:", value=response, height=100, max_chars=None, key=f"chatbot_response_{counter}")

        if response.lower() in ['goodbye', 'bye']:
            st.write("Thank you for chatting with me. Have a great day!")
            st.stop()

if __name__ == '__main__':
    main()

2024-07-21 13:54:44.262 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-07-21 13:54:44.266 Session state does not function when running a script without `streamlit run`


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.212.28:8501

